In [1]:
from binance.client import Client

In [ ]:
client = Client("0ppfXZtnEyP5tINDcKvVSh5XLwPNHChrS7MTQ44MgS7zVXEeO1QqpICaHBr94bGa", "LyAdCy4nMkwSOGVf7lAdwjXnI7ijPoLn6WiPOMhFwn8Lc8hYvn4VtKh3755AW6aA")